# Creación contenedor

## Instalación docker en linux

Las instrucciones las podemos encontrar en la página de docker.

Enlace: https://docs.docker.com/desktop/install/ubuntu/

## Instalación docker en Windows

Descargamos el ejecutable de la página oficial.

Enlace: https://docs.docker.com/desktop/install/windows-install/

## Descarga imagen inicial 

Descargamos la imagen de ubuntu con el siguiente comando.

```bash
docker pull ubuntu:22.04
```

Crear y ejecutar un contenedor interactivo

```bash
docker run -it --gpus all --name torchmd-config ubuntu:22.04 /bin/bash
```


El parámetro `-it` indica que el contenedor se ejecutará en modo interactivo.

`--gpus all` le da acceso al contenedor a las gpus, en este caso todas

`--name torchmd-config` le da un nombre al contenedor, en este caso, torchmd-config.

`ubuntu:22.04` es el nombre de la imagen.

`/bin/bash` especifica que queremos utilizar el shell Bash dentro del contenedor.

## Instalación dependencias básicas

### Actualizar la imagen

```bash
apt update
```

### Git

```bash
apt install git
```

### Curl

```bash
apt install curl
```

### Mamba

Nos dirigimos al directorio /tmp

```bash
cd /tmp
```

Descargamos el ejecutable de Mamba

```bash
curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
```

Lo instalamos

```bash
bash Miniforge3-$(uname)-$(uname -m).sh
```

Cuando nos pregunte lo siguiente, seleccionamos yes.

Do you wish to update your shell profile to automatically initialize conda?
This will activate conda on startup and change the command prompt when activated.
If you'd prefer that conda's base environment not be activated on startup,
   run the following command when conda is activated:

conda config --set auto_activate_base false

You can undo this by running `conda init --reverse $SHELL`? [yes|no]
[no] >>> 

### Reinicio del contenedor

Nos salimos del contenedor con el comando

```bash
exit
```

Detenemos el contenedor con el comando

```bash
docker stop torchmd-config
```

Iniciamos el contenedor con el comando
```bash
docker start torchmd-config
```

Entramos al contenedor con el comando

```bash
docker exec -it torchmd-config /bin/bash
```

## Instalación de la arquitectura torchmd-net

Nos dirigimos a la carpeta /temp con el comando.

```bash
cd /tmp
```

Descargamos el repositorio de GitHub

```bash
git clone https://github.com/torchmd/torchmd-net.git
```

Vamos a la carpeta del repositorio
```bash
cd torchmd-net
```

Creamos el ambiente virtual con el comado

```bash
mamba env create -f environment.yml
```

Lo activamos con
```bash
mamba activate torchmd-net
```

## Habilitar uso de gpu

Dentro del ambiente virtual corroboramos nuestra versión de cuda con

```bash
python -c "import torch; print(torch.version.cuda)"
```

### Si CUDA>=12

Instalamos las dependencias con

```bash
conda install -c conda-forge cuda-nvcc cuda-libraries-dev cuda-version "gxx<12" pytorch=*=*cuda*
```

### Si CUDA<12

Instalamos las dependencias con

```bash
conda install -c nvidia "cuda-nvcc<12" "cuda-libraries-dev<12" "cuda-version<12" "gxx<12" pytorch=*=*cuda*
```

## Instalar codigo arquitectura torchmd-net en el ambiente virtual

Para poder ejecutar torchmd-net en terminal debemos instalarlo, en el directorio /tmp/torchmd-net ejecutamos el comando.

```bash
pip install -e .
```

## Instalar Jupyter notebook

Dentro del ambiente virtual intalamos jupyter notebook e ipykernel con los comandos

```bash
pip install notebook
pip install ipykernel
```

### Registar ambiente virtual como ipykernel

```bash
python -m ipykernel install --user --name=torchmd-net
```

## Subir contenedor a docker hub

```bash
docker commit fb95480f4d39 emmanuelzula/tesis:0.1

docker push emmanuelzula/tesis:0.1
```

## Configuración de funcionalidad

Creación del archivo DockerFile dondé se encontrará la configuración

```DockerFile
# Usa una imagen base de Ubuntu
FROM emmanuelzula/tesis:0.1

# Documenta la intención de exponer el puerto 8888, pero no lo publica por sí mismo
# Recuerda usar -p al ejecutar el contenedor para mapear el puerto de host al contenedor
EXPOSE map[8888/tcp:{}]

# Establece el directorio de trabajo en /workspace
WORKDIR /workspace

# Ejecuta un comando durante la construcción de la imagen
# Agrega una línea al final de ~/.bashrc para activar el ambiente virtual 'torchmd-net'
RUN /bin/sh -c echo "mamba activate torchmd-net" >> ~/.bashrc # buildkit


# Comando por defecto al iniciar el contenedor
# Activa automáticamente el ambiente virtual al iniciar el contenedor
CMD ["bash"]
```

En el directorio donde se encuentra el Dockerfile ejecutamos.
```bash
docker build -t nombre-de-la-imagen:etiqueta .
```

Corroboramos que la imagen se haya creado correctamente con el comando.
```bash
docker images
```

Ejecutamos el contenedor localmente
```bash
docker run -it nombre-de-la-imagen:etiqueta
```

Iniciamos seción para poder subir el contenedor a DockerHub
```bash
docker login
```

Etiquetamos la imagen local
```bash
docker tag nombre-de-la-imagen:etiqueta nombre-de-usuario/nombre-de-la-imagen:etiqueta
```

Subimos la imagen a DockerHub
```bash
docker push nombre-de-usuario/nombre-de-la-imagen:etiqueta
```

## Instalación del contenedor

```bash
docker run -it --gpus all -p 9999:8888 -v $PWD:/workspace --name torchmd-net-tesis --shm-size 16G emmanuelzula/tesis:0.1 /bin/bash
```

Este comando se utiliza para ejecutar un contenedor Docker con ciertas configuraciones.

1. **docker run**: Esto es el comando principal de Docker para ejecutar un contenedor.

2. **-it**: Estos son argumentos que se utilizan para indicar que se desea una terminal interactiva. Esto permite interactuar con el contenedor a través de la línea de comandos.

3. **--gpus all**: Indica que deseas asignar todos los recursos de GPU disponibles en el contenedor. Esto asume que tienes GPU y que has configurado Docker para admitir GPU.

4. **-p 9999:8888**: Esto mapea el puerto 8888 del contenedor al puerto 9999 de tu host local. Significa que si el contenedor ejecuta un servicio en el puerto 8888, podrás acceder a él desde tu navegador en `localhost:9999`.

5. **-v $PWD:/workspace**: Este argumento establece un volumen (mount) que vincula el directorio actual (`$PWD`) en tu host local con el directorio `/workspace` en el contenedor. Esto permite compartir archivos y datos entre tu sistema local y el contenedor.

6. **--name torchmd-net**: Asigna un nombre al contenedor, en este caso, "torchmd-net". Puedes usar este nombre para hacer referencia al contenedor en lugar de su identificador largo.

7. **--shm-size 16G**: Esto configura el tamaño de la memoria compartida (shared memory) dentro del contenedor en 16 gigabytes. Algunas aplicaciones pueden requerir más memoria compartida.

8. **emmanuelzula/servicio:1.3**: Es la imagen de Docker que se utilizará para crear el contenedor. En este caso, se está utilizando la imagen "emmanuelzula/servicio" con la etiqueta "1.3".

9. **/bin/bash**: Es el comando que se ejecutará dentro del contenedor. En este caso, se inicia un shell Bash dentro del contenedor, lo que te permite interactuar con él.